## Foundations

This is key - experiment with this notebook, and learn Ref [Lesson 14](https://youtu.be/veqj0DsZSXU?feature=shared&t=5283). 

In [1]:
import torch
import matplotlib.pyplot as plt
import random

If you're not familiar with the pieces - like try-except blocks, `getattr()`, and decorators, you won't understand how those pieces fit together. Work on that, and you will start to understand how it all fits together. Trust the process.

The goal of this is to understand the components of the Learner class, which we'll be building next. It is a general purpose training loop, which we can use to do what we want it to do. To make that happen, we need to use callbacks - so we work on those first.

## Callbacks

### Callbacks as GUI events

In [2]:
import ipywidgets as widgets

From the [ipywidget docs](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html):

- *the button widget is used to handle mouse clicks. The on_click method of the Button can be used to register function to be called when the button is clicked*

In [8]:
w = widgets.Button(description='Click me')

In [9]:
w

Button(description='Click me', style=ButtonStyle())

In [10]:
def f(o): print('hi')

In [11]:
w.on_click(f)

This assigns the f function to the `on_click()` callback

In [12]:
w

Button(description='Click me', style=ButtonStyle())

*NB: When callbacks are used in this way they are often called "events".*

### Creating your own callback

That was an existing callback - let's create our own!

In [13]:
from time import sleep

In [14]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [15]:
slow_calculation()

30

For such an operation that is slow (like training a model!), would be nice to print out a progress bar. 

For this you define a `cb=None` that you call, at specific intervals.

In [16]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb: cb(i)
    return res 

In [17]:
def show_progress(epoch): print(f"Awesome! We've finished epoch {epoch}!")

In [18]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

### Lambdas and partials

How do you do it using lambdas?

In [19]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

Now let's try and make it more sophisticated.

In [20]:
def show_progress(exclamation, epoch): print(f"{exclamation}! We've finished epoch {epoch}!")

In [21]:
slow_calculation(lambda o: show_progress("OK I guess", o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [22]:
def make_show_progress(exclamation):
    return lambda epoch: print(f"{exclamation}! We've finished epoch {epoch}!")

In [23]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

Now we can convert that lambda expression into a function definition, as we did before. Same thing:

In [24]:
def make_show_progress(exclamation):
    def _inner(epoch): print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

In [25]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

We can do exactly the same thing with partial;

In [26]:
from functools import partial

In [27]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

We tend to use partial a lot, so it's good to practice this.

In [28]:
f2 = partial(show_progress, "OK I guess")

### Callbacks as callable classes

cb does not have to be function; it just has to be a callable. One other way to create the callable is to use `__call__`:

In [31]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"): self.exclamation = exclamation
    def __call__(self, epoch): print(f"{self.exclamation}! We've finished epoch {epoch}!")

Then create an object which is callable:

In [32]:
cb = ProgressShowingCallback("Just super")

In [33]:
slow_calculation(cb)

Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

Works the same way!!

make sure to really get familiar with the basics of the concepts above. Comes up all the time in PyTorch code, and in the code we will be writing.

### Multiple callback funcs; `*args` and `**kwargs`

In [34]:
def f(*a, **b): print(f"args: {a}; kwargs: {b}")

In [35]:
f(3, 'a', thing1="hello")

args: (3, 'a'); kwargs: {'thing1': 'hello'}


Arguments passed by position, like `3` or `'a'`, are called `args`, and those passed by `=` are called `kwargs`. The former is a tuple, the latter takes the form of a dictionary.

Now let's say we have this function:

In [36]:
def g(a,b,c=0): print(a,b,c)

In [37]:
args = [1,2]
kwargs = {'c':3}
g(*args, **kwargs)

1 2 3


Note that the elements of `args = [1,2]` get passed in as individual arguments, and the `{'c':3}` is passed as a value! Also note, it's the `*` and `**` doing the trick here.

Now here's a new, cool way of doing callbacks:

In [38]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

Note how above there are `before_calc` and `after_calc` methods defined. Now the callback will be a class with `before_calc` and `after_calc` methods defined. 

In [39]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc (self, *args, **kwargs): print(f"Done step")

In [40]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [41]:
class PrintStatusCallback():
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f"About to start: {epoch}")
    def after_calc (self, epoch, val, **kwargs): print(f"After {epoch}: {val}")

In [42]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

So - the callback doesn't just have to be functions or callables - the callback can contain methods. Note how the `cb: cb.after_calc(i, val=res)` passes in value afte the calculation, but we just ignore it using the `def after_calc (self, *args, **kwargs): print(f"Done step")` code, and include it in the `def after_calc (self, epoch, val, **kwargs): print(f"After {epoch}: {val}")` code.

In [43]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc (self): print(f"Done step")

In [45]:
slow_calculation(PrintStepCallback())

About to start


TypeError: after_calc() got an unexpected keyword argument 'val'

See? That won't work.

### Modifying behavior

In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb,'before_calc'): cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb,'after_calc'):
            if cb.after_calc(i, res):
                print("stopping early")
                break
    return res

In [ ]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f"After {epoch}: {val}")
        if val>10: return True

In [ ]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

In [ ]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb,self.res = cb,0
    
    def callback(self, cb_name, *args):
        if not self.cb: return
        cb = getattr(self.cb,cb_name, None)
        if cb: return cb(self, *args)

    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i*i
            sleep(1)
            if self.callback('after_calc', i):
                print("stopping early")
                break

In [ ]:
class ModifyingCallback():
    def after_calc (self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2

In [ ]:
calculator = SlowCalculator(ModifyingCallback())

In [ ]:
calculator.calc()
calculator.res

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


15

## `__dunder__` thingies

Anything that looks like `__this__` is, in some way, *special*. Python, or some library, can define some functions that they will call at certain documented times. For instance, when your class is setting up a new object, python will call `__init__`. These are defined as part of the python [data model](https://docs.python.org/3/reference/datamodel.html#object.__init__).

For instance, if python sees `+`, then it will call the special method `__add__`. If you try to display an object in Jupyter (or lots of other places in Python) it will call `__repr__`.

In [ ]:
class SloppyAdder():
    def __init__(self,o): self.o=o
    def __add__(self,b): return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self): return str(self.o)

In [ ]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

Special methods you should probably know about (see data model link above) are:

- `__getitem__`
- `__getattr__`
- `__setattr__`
- `__del__`
- `__init__`
- `__new__`
- `__enter__`
- `__exit__`
- `__len__`
- `__repr__`
- `__str__`

READ the above methods. Done yet?

### `__getattr__` and `getattr`

Here's a class, with just 3 attributes.

In [ ]:
class A: a,b=1,2

In [ ]:
a = A()

In [ ]:
a.b

2

In [ ]:
getattr(a, 'b')

2

In [ ]:
getattr(a, 'b' if random.random()>0.5 else 'a')

2

Ultimately, what is being called is `__getattr__`:

In [ ]:
class B:
    a,b=1,2
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello from {k}'

In [ ]:
b = B()

In [ ]:
b.a

1

In [ ]:
b.foo

'Hello from foo'